In [3]:
from google.colab import drive
!pip install torch_scatter
!pip install torch_geometric

drive.mount('/content/drive')

# cd into where the data file is stored
%cd /content/drive/MyDrive/cs224w-project/

!pip install python-chess

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/cs224w-project
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/cs224w-project


In [20]:
#Extract Features
import chess
import torch

def calculate_material_balance(board):
    piece_values = {chess.PAWN: 1, chess.KNIGHT: 3, chess.BISHOP: 3, chess.ROOK: 5, chess.QUEEN: 9, chess.KING: 0}
    material_balance = 0
    for piece_type in piece_values:
        material_balance += len(board.pieces(piece_type, chess.WHITE)) * piece_values[piece_type]
        material_balance -= len(board.pieces(piece_type, chess.BLACK)) * piece_values[piece_type]
    return material_balance

def calculate_pawn_structure(board):
    # For simplicity, just counting the number of pawns for now.
    white_pawns = len(board.pieces(chess.PAWN, chess.WHITE))
    black_pawns = len(board.pieces(chess.PAWN, chess.BLACK))
    return white_pawns - black_pawns

def calculate_mobility(board):
    return board.legal_moves.count()

def get_squares_around(square):
    """ Get the squares around a given square. """
    surrounding_squares = []
    file = chess.square_file(square)
    rank = chess.square_rank(square)

    for f in range(file - 1, file + 2):  # File of the square and its adjacent files
        for r in range(rank - 1, rank + 2):  # Rank of the square and its adjacent ranks
            if 0 <= f <= 7 and 0 <= r <= 7:  # Ensure the file and rank are within bounds
                adjacent_square = chess.square(f, r)
                if adjacent_square != square:
                    surrounding_squares.append(adjacent_square)

    return surrounding_squares

def evaluate_king_safety(board):
    """ Evaluate the safety of the king based on surrounding pawns. """
    king_safety = 0
    for color in [chess.WHITE, chess.BLACK]:
        king_square = board.king(color)
        pawn_squares = board.pieces(chess.PAWN, color)
        safety_count = sum(1 for sq in get_squares_around(king_square) if sq in pawn_squares)
        king_safety += safety_count if color == chess.WHITE else -safety_count
    return king_safety



def control_of_center(board):
    center_squares = [chess.E4, chess.D4, chess.E5, chess.D5]
    center_control = 0
    for square in center_squares:
        if board.is_attacked_by(chess.WHITE, square):
            center_control += 1
        if board.is_attacked_by(chess.BLACK, square):
            center_control -= 1
    return center_control

def piece_development(board):
    """ Calculate the development of pieces from their starting positions. """
    development = 0

    # Initial positions for non-pawn pieces except the king.
    initial_positions = {
        chess.WHITE: [chess.A1, chess.B1, chess.C1, chess.D1, chess.E1, chess.F1, chess.G1, chess.H1],
        chess.BLACK: [chess.A8, chess.B8, chess.C8, chess.D8, chess.E8, chess.F8, chess.G8, chess.H8]
    }

    # Count developed pieces (those not in their initial positions).
    for color in [chess.WHITE, chess.BLACK]:
        for piece_type in [chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN]:
            for square in board.pieces(piece_type, color):
                if square not in initial_positions[color]:
                    development += 1 if color == chess.WHITE else -1

    return development


def extract_features(fen):
    board = chess.Board(fen)
    features = {
        "material_balance": calculate_material_balance(board),
        "pawn_structure": calculate_pawn_structure(board),
        "mobility": calculate_mobility(board),
        "king_safety": evaluate_king_safety(board),
        "center_control": control_of_center(board),
        "piece_development": piece_development(board)
    }

    features_vector = [features["material_balance"], features["pawn_structure"],
                    features["mobility"], features["king_safety"],
                    features["center_control"], features["piece_development"]]

    return torch.tensor(features_vector)

In [11]:
#pre-process
import chess
import pickle
import warnings
import concurrent.futures

#from extract_features import extract_features
import requests

from tqdm import tqdm

DEPTH = 5
MODE = 'eval'

def convert_moves_to_san(moves_str):
    # Split the string into individual moves, ignoring the white and black move indicators
    moves = moves_str.split()
    san_moves = []

    # Extract the standard algebraic notation (SAN) part of each move (ignoring the move number and color indicator)
    for move in moves:
        san_move = move.split('.')[1]  # Take only the move part after the dot
        san_moves.append(san_move)

    # Combine the moves into a single space-separated string
    san_moves_str = ' '.join(san_moves)
    return san_moves_str

def moves_to_fen(moves):
    board = chess.Board()
    fen_list = [board.fen()]  # Include the starting position

    for move_san in moves.split():
        move = board.parse_san(move_san)
        board.push(move)
        fen_list.append(board.fen())

    return fen_list

def preprocess_chess_data(file_path):
    # List to hold all games moves
    games_moves = []

    # Open the file and skip the first five lines of metadata
    with open(file_path, 'r') as file:
        for _ in range(5):
            next(file)  # Skip metadata lines

        # Process each game entry line
        for line in file:
            if '###' in line:
                # Split the line at '###' and take the second part, which contains the moves
                moves = line.split('###')[1].strip()
                # Store the moves in the list
                games_moves.append(moves)

    return games_moves

# Process the data to convert moves to FEN for each game
def games_to_fen(games_moves):
    all_games_fen = []
    for moves_str in games_moves:
        san_moves = convert_moves_to_san(moves_str)
        game_fen = moves_to_fen(san_moves)
        all_games_fen.append(game_fen)
    return all_games_fen


def create_feature_vector(fen):
    features = extract_features(fen)
    features_vector = [features["material_balance"], features["pawn_structure"],
                       features["mobility"], features["king_safety"],
                       features["center_control"], features["piece_development"]]

    return features_vector


def fetch_label(fen):
    """
    Fetch the label for a given FEN from the API.
    """
    try:
        response = requests.get(f'https://stockfish.online/api/stockfish.php?fen={fen}&depth={DEPTH}&mode={MODE}')
        if response.status_code == 200:
            return response.json().get('data').split()[2], False
        else:
            print(f"Error fetching data for FEN: {fen}. Status Code: {response.status_code}")
            return None, True
    except requests.RequestException as e:
        print(f"Request failed for FEN: {fen}. Error: {e}")
        return None, True


def process_chess_data(file_path):
    chess_games_moves = preprocess_chess_data(file_path)
    subset_games_fen = games_to_fen(chess_games_moves)

    # all_games = []
    all_labels = []
    exception_indices = []

    for game in tqdm(subset_games_fen):
        for i, fen in enumerate(game):
    #         x = [create_feature_vector(fen) for fen in game]
            labels, exception_occurred = fetch_label(fen)

    #         all_games.append(x)
            all_labels.append(labels)

            if exception_occurred:
                exception_indices.append(i)

    # data = {'x': all_games, 'y': all_labels}

    data = {'fen': subset_games_fen, 'y': all_labels}

    return data, exception_indices


#if __name__ == '__main__':
    file_path = 'data/subset.txt'
    data, exception_indices = process_chess_data(file_path)
    pickle.dump(data, open('/content/drive/MyDrive/cs224w-project/subset_games_test.pkl', 'wb'))

In [17]:
#graph
import torch
from torch_geometric.data import Data
import pickle

games_fen = pickle.load(open('/content/drive/MyDrive/cs224w-project/subset_games_fen.pkl', 'rb'))

# A function to encode a FEN string into a numerical feature vector
def encode_fen(fen):
    # Initialize a binary vector for the board
    # 12 piece types (6 for white, 6 for black) x 64 squares
    board_vector = [0] * (12 * 64)

    # Define a mapping from piece symbols to their index offsets
    piece_to_index = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }

    # Fill the vector with the presence of pieces
    pieces = fen.split(' ')[0]  # Get the piece placement part of the FEN
    square = 0
    for char in pieces:
        if char.isdigit():  # Empty squares
            square += int(char)
        elif char in piece_to_index:  # Occupied square
            index = piece_to_index[char] * 64 + square
            board_vector[index] = 1
            square += 1
        elif char == '/':  # New rank
            continue

    return board_vector


# Encode all FENs and create a mapping from FEN to node index
fen_to_index = {}
node_features = []

for game_fens in games_fen:
    for fen in game_fens:
        if fen not in fen_to_index:
            fen_to_index[fen] = len(fen_to_index)
            encoded_fen = encode_fen(fen)
            node_features.append(encoded_fen)

# Convert node features to a tensor
node_features_tensor = torch.tensor(node_features, dtype=torch.float)

# Create edges based on the sequence of moves
edge_indices = []

for game_fens in games_fen:
    for i in range(len(game_fens) - 1):
        source = fen_to_index[game_fens[i]]
        target = fen_to_index[game_fens[i + 1]]
        edge_indices.append((source, target))

# Convert edge indices to a tensor
edge_index_tensor = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()

# Create a PyG Data object
data = Data(x=node_features_tensor, edge_index=edge_index_tensor)


import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

# Convert PyG Data object to NetworkX graph
G = to_networkx(data, to_undirected=True)

# Draw the graph using NetworkX
#plt.figure(figsize=(12, 8))
#nx.draw(G, with_labels=False, node_color='lightblue', node_size=25, edge_color='gray')
#plt.title('Graph Representation of Chess Games')
#plt.show()

In [37]:
#model
import pickle
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GATConv
import torch_scatter

#from preprocess import process_chess_data
#from extract_features import extract_features

import time
from tqdm import tqdm

class GATChessModel(torch.nn.Module):
    def __init__(self, num_features, heads=8, dropout=0.6):
        super(GATChessModel, self).__init__()
        self.conv1 = GATConv(num_features, 32, heads=heads, dropout=dropout)
        self.conv2 = GATConv(32 * heads, 1, heads=1, dropout=dropout)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)

        return x


def get_edge_indices():
    file_path = '/content/drive/MyDrive/cs224w-project/subset.txt'
    # _, exception_indices = process_chess_data(file_path)
    exception_indices = []

    # Read exception indices from file
    with open('/content/drive/MyDrive/cs224w-project/exception_indices.txt', 'r') as file:
        for line in file:
            exception_indices.append(int(line.strip()))

    print("exceptions", exception_indices)

    with open('/content/drive/MyDrive/cs224w-project/subset_games_test.pkl', 'rb') as f:
        node_id = 0
        fen_to_node = {}

        games = pickle.load(f)
        print("NUM GAME", len(games))
        for i, game in enumerate(games):
            if i in exception_indices:  # skip games with exceptions
                continue
            for state in game:
                if state not in fen_to_node:
                    fen_to_node[state] = node_id
                    node_id += 1

        print("NODE ID", node_id)

        src, target = [], []
        for j, game in enumerate(games):
            for i in range(len(game) - 1):
                if j in exception_indices:  # skip games with exceptions
                    continue
                else:
                    src.append(fen_to_node[game[i]])
                    target.append(fen_to_node[game[i+1]])

        edge_index = torch.tensor([src, target], dtype=torch.long)

        return edge_index


def get_data():
    from sklearn.model_selection import train_test_split
    with open('/content/drive/MyDrive/cs224w-project/subset_games_test.pkl', 'rb') as f:
        data = pickle.load(f)
        print(extract_features(data['fen'][0][0]))
        x = [extract_features(fen) for game in data['fen'] for fen in game]
        x = torch.tensor(torch.stack(x), dtype=torch.float32)
        y = [float(score) for game in data['y'] for score in game]
        edge_index = torch.tensor(get_edge_indices(), dtype=torch.long)

        y = torch.tensor(y, dtype=torch.float32)

        dataset = Data(x=x, edge_index=edge_index, y=y)

        # Split the dataset into training, validation, and testing sets
        x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=42)
        x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

        # Create Data objects for each set
        train_data = Data(x=x_train, edge_index=edge_index, y=y_train)
        val_data = Data(x=x_val, edge_index=edge_index, y=y_val)
        test_data = Data(x=x_test, edge_index=edge_index, y=y_test)

        return train_data, val_data, test_data


if __name__ == '__main__':
    num_features = 6
    learning_rate = 0.01
    num_epochs = 100

    model = GATChessModel(num_features)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_func = torch.nn.MSELoss()  # Mean Squared Error for regression

    data, val_data, test_data = get_data()

    model.train()
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        start_time = time.time()

        optimizer.zero_grad()
        out = model(data)
        loss = loss_func(out, data.y)
        loss.backward()
        optimizer.step()

        end_time = time.time()
        epoch_duration = end_time - start_time
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Duration: {epoch_duration:.2f} sec")

        # print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    # Save the trained model
    torch.save(model.state_dict(), '/content/drive/MyDrive/cs224w-project/trained_model.pth')

    model = GATChessModel(num_features)
    model.load_state_dict(torch.load('/content/drive/MyDrive/cs224w-project/trained_model.pth'))
    model.eval()

    with torch.no_grad():
        model_output = model(test_data)



tensor([ 0,  0, 20,  0,  0,  0])


<ipython-input-37-b0393da41216>:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(torch.stack(x), dtype=torch.float32)
<ipython-input-37-b0393da41216>:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(get_edge_indices(), dtype=torch.long)


exceptions []
NUM GAME 2
NODE ID 4


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3381])) that is different to the input size (torch.Size([3381, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epochs:   1%|          | 1/100 [00:00<00:11,  8.54it/s]

Epoch 1/100, Loss: 268.2765, Duration: 0.12 sec


...

Epoch 98/100, Loss: 7.8442, Duration: 0.13 sec


Epochs:  99%|█████████▉| 99/100 [00:13<00:00,  7.39it/s]

Epoch 99/100, Loss: 7.8737, Duration: 0.13 sec


Epochs: 100%|██████████| 100/100 [00:13<00:00,  7.33it/s]

Epoch 100/100, Loss: 7.8312, Duration: 0.13 sec


In [41]:
    import numpy as np

    true_labels = test_data.y.numpy()
    predictions = model_output.argmax(dim=1).numpy()

    result = (true_labels - predictions)**2

    # Evaluate results (Mean Squared Error and Mean Absolute Error)
    mse = np.mean((result)**2)
    mae = np.mean(np.abs(true_labels - predictions))

    print("Mean Squared Error:", mse)
    print("Mean Absolute Error:", mae)

Mean Squared Error: 501.4729558698046
Mean Absolute Error: 1.6331724122038176
